In [1]:
from functools import cache
from aocutils.common import flatten, zippify
from itertools import product
import networkx as nx
from collections import defaultdict
inf = float('inf')

In [2]:
   
@cache
def direcnext(start, goal):
    # goes to a button and back to A
    
    keys = {'^':(0,1), 'A':(0,2), '<':(1,0), 'v':(1,1), '>':(1,2)}
    
    # print('pressing from', loc, 'goal', keys[char])
    G = nx.Graph()
    for loc1 in keys.values():
        for loc2 in keys.values():
            if abs(loc1[0]-loc2[0]) + abs(loc1[1]-loc2[1]) == 1:
                G.add_edge(loc1, loc2)
    options = []
    return tuple(nx.all_shortest_paths(G, keys[start], keys[goal]))
        
        # for waytoA in nx.all_shortest_paths(G, keys[goal], keys['A']):
            # options.append(path + ['A'] + waytoA[1:])
    return options

@cache 
def numericnext(start, goal):
    keys = {'7':(0,0), '8':(0,1), '9':(0,2), '4':(1,0), '5':(1,1), '6': (1,2), '1':(2,0), '2':(2,1), '3':(2,2), '0':(3,1), 'A':(3,2)}
    # gives back options to next step
    G = nx.Graph()
    for loc1 in keys.values():
        for loc2 in keys.values():
            if abs(loc1[0]-loc2[0]) + abs(loc1[1]-loc2[1]) == 1:
                G.add_edge(loc1, loc2)
    return tuple(nx.all_shortest_paths(G, keys[start], keys[goal]))

def numerictotal(goalstring):
    # given a string, comes up with paths how to type in that string
    paths = [[]]
    for a,b in zippify('A'+goalstring, 2):
        newpaths = []
        for p1 in paths:
            for p2 in numericnext(a,b):
                newpaths.append(p1.copy() + p2.copy()[1:] + ['press'])
        paths = newpaths
    return tuple(paths)
def directtotal(goalstring):
    # given instructions for a directional robot, comes up with paths how to type in that string
    paths = [[]]
    
    for a,b in zippify(['A']+goalstring, 2):
        newpaths = []
        if a == b:
            for p1 in paths:
                newpaths.append(p1.copy() + ['press'])
        else:
            for p2 in direcnext(a,b):
                # print('from to', a,b,'path', p2)
                for p1 in paths:
                    newpaths.append(p1.copy() + p2.copy()[1:] + ['press'])
        paths = newpaths
    return paths
            
        
def numericpath_to_direcpath(path):
    # this essentially gives the goal for the next robot
    needed = []
    a = (3,2)
    for b in path:
        if b == 'press': needed.append('A')
        else:
            assert abs(a[0] - b[0]) + abs(a[1] - b[1]) == 1
            if a[0] - b[0] == 1: needed.append('^')
            elif a[0] - b[0] == -1:needed.append('v')
            elif a[1] - b[1] == 1:needed.append('<')
            elif a[1] - b[1] == -1:needed.append('>')
            else: print('error')
            a = b
    return (needed)

def direcpath_to_directpath(path):
    # this essentially gives the goal for the next robot
    needed = []
    a = (0,2)
    for b in path:
        if b == 'press': needed.append('A')
        else:
            assert abs(a[0] - b[0]) + abs(a[1] - b[1]) == 1
            if a[0] - b[0] == 1: needed.append('^')
            elif a[0] - b[0] == -1:needed.append('v')
            elif a[1] - b[1] == 1:needed.append('<')
            elif a[1] - b[1] == -1:needed.append('>')
            else: print('error')
            a = b
    return (needed)
 


In [3]:
mapping = {}
def parse(ins, ins2):
    global mapping
    cur = 'A'
    p2 = 0
    for ch in ins:
        if cur == ch:
            newp2 = p2
            mapping[(cur, ch)] = ['A']
        else:
            newp2 = ins2.index('A', p2+1)
            new = ins2[p2:newp2+1]
            if (cur,ch) in mapping and mapping[(cur, ch)] != new:
                print('value already in from', cur, 'to', ch, new, '!=', mapping[(cur, ch)])
            mapping[(cur, ch)] = new
        # print(cur, ch,ins2[p2:newp2+1], newp2)
        p2 = newp2+1
        cur = ch
        
parse(list('<A^A>^^AvvvA'),list('v<<A>>^A<A>AvA<^AA>A<vAAA>^A'))
mapping


# this code is to fill the mapping that is used later on
f = open("in.txt").read().split("\n")
lines = [line for line in f]
lines[0]

mapping = {}
answers = defaultdict(lambda: defaultdict(lambda: int(99999999)))
for comb in list(product(list('<>^vA'), repeat = 2)):
    nextlevel = directtotal(list(comb))
    for option in nextlevel:
        ins1 = direcpath_to_directpath(option)
        for option2 in directtotal(ins1):
            ins2 = direcpath_to_directpath(option2)
            for option3 in directtotal(ins2):
                ins3 = direcpath_to_directpath(option3)
                answers[comb][tuple(ins1)] = min(answers[comb][tuple(option)], len(ins3))
                # for option4 in directtotal(ins3):
                #     ins4 = direcpath_to_directpath(option4)
for c in answers:
    best = min(v for k,v in answers[c].items())
    cnt = sum(1 for k,v in answers[c].items() if v == best)
    candidates = [k for k,v in answers[c].items() if v == best]
    if len(candidates) > 1:
        print(c,candidates, 'only the first value is correct')
    parse(c, candidates[0])
    


('v', 'A') [('<', 'v', 'A', '^', '>', 'A'), ('<', 'v', 'A', '>', '^', 'A')] only the first value is correct


In [4]:
# these are the keypresses robot 1 needs to do
# works on example but not on 25 robots
f = open("in.txt").read().split("\n")
lines = [line for line in f]
lines[0]

@cache
def explode(smallpart):
    i = smallpart
    smallpart = flatten([mapping[a] for a in zippify((smallpart))])
    for step in range(14):
        smallpart = flatten([mapping[a] for a in zippify(['A']+list(smallpart))])
    
    return len(smallpart)

ans = 0
for line in lines:
    minn = inf
    for p0 in numerictotal(line):
        ins = numericpath_to_direcpath(p0)
        for step in range(10):
            ins = flatten([mapping[a] for a in zippify(['A']+ins,2)])
        # minn = min(len(ins), minn)

        temp = 0
        for smallpart in zippify(['A']+ins,2):
            temp += explode(smallpart)
        minn = min(minn, temp)
            
    print('found', minn, line[:-1])
    ans += minn * int(line[:-1])
ans
    

found 87288844796 463
found 84248089342 340
found 90594397580 129
found 80732180764 083
found 90750571880 341


118392478819140